In [1]:
pip install openai

Note: you may need to restart the kernel to use updated packages.


In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from openai import OpenAI
import random
from deep_translator import GoogleTranslator
import time

In [3]:
df = pd.read_csv('raw_del2.csv') # изменить путь
# df = pd.read_json('train_hackathon_dataset.json')
# df = pd.read_csv('raw_precleaned.csv')
# df = pd.read_csv('LLMaugmented_with_raw_tags.csv')

In [2]:
tag_dict = {'[PROJECT]': 'проект_тег',
 '[TYPE]': 'тип_тег',
 '[SIZE]': 'размер_тег',
 '[НОМЕР_ОБЪЕКТА]': 'номер_объекта_тег',
 '[REGIONAL_OFFICE]': 'региональный_офис_тег',
 '[NUMBER]': 'число_тег',
 '[MUNICIPAL_DISTRICT]': 'муниципальный_округ_тег',
 '[OGRN]': 'огрн_тег',
 '[CITY]': 'город_тег',
 '[TIME]': 'время_тег',
 '[DIMENSIONS]': 'размеры_тег',
 '[NUMBER_OF_PAGES]': 'количество_страниц_тег',
 '[PRODUCT]': 'продукт_тег',
 '[SIGNATURE]': 'подпись_тег',
 '[SERVICE_AREA]': 'зона_обслуживания_тег',
 '[LAW_NUMBER]': 'номер_закона_тег',
 '[QUANTITY]': 'количество_тег',
 '[PAGE]': 'страница_тег',
 '[PRODUCT_MARK]': 'марка_продукта_тег',
 '[PROJECT_NUMBER]': 'номер_проекта_тег',
 '[EXECUTIVE_DIRECTOR]': 'исполнительный_директор_тег',
 '[TEMPERATURE]': 'температура_тег',
 '[DOMAIN]': 'домен_тег',
 '[FILIAL]': 'филиал_тег',
 '[DATE]': 'дата_тег',
 '[KPP]': 'кпп_тег',
 '[VOLUME]': 'объем_тег',
 '[RESOURCE_TYPES]': 'типы_ресурсов_тег',
 '[SKVAGH_NUMBER]': 'номер_скваха_тег',
 '[DIAGNOSIS]': 'диагностика_тег',
 '[PAGE_NUMBER]': 'номер_страницы_тег',
 '[FRACTION]': 'фракция_тег',
 '[WORK_TYPE]': 'тип_работы_тег',
 '[ELECTRONIC_SIGNATURE]': 'электронная_подпись_тег',
 '[DOCUMENT_SIGNATURE]': 'подпись_документа_тег',
 '[NETWORK]': 'сеть_тег',
 '[NAME_OF_FIELD]': 'название_поля_тег',
 '[DISTANCE]': 'расстояние_тег',
 '[AUTONOMOUS_REGION]': 'автономная_область_тег',
 '[URL]': 'url-адрес_тег',
 '[DISTRICT]': 'округ_тег',
 '[DIRECTOR]': 'директор_тег',
 '[PLACE]': 'место_тег',
 '[PLAN]': 'план_тег',
 '[CHEMICAL_REAGENT]': 'химический_реагент_тег',
 '[WWW]': 'www_тег',
 '[TITLE]': 'заголовок_тег',
 '[PROJECT_NAME]': 'название_проекта_тег',
 '[PHONE_PROVIDER]': 'телефонный_провайдер_тег',
 '[SYSTEM]': 'система_тег',
 '[REGION]': 'область_тег',
 '[ID]': 'идентификатор_тег',
 '[LICENSE]': 'лицензия_тег',
 '[CURRENCY]': 'валюта_тег',
 '[BIC]': 'бик_тег',
 '[PIPELINE]': 'трубопровод_тег',
 '[AREA]': 'область_тег',
 '[RIVER]': 'река_тег',
 '[DEPOSITS]': 'депозиты_тег',
 '[ЛОКАЦИЯ]': 'локация_тег',
 '[ACTIVITY]': 'активность_тег',
 '[LENGTH]': 'длина_тег',
 '[DELIVERY_DATE]': 'дата_доставки_тег',
 '[BANK_ID]': 'идентификатор_банка_тег',
 '[YEAR]': 'год_тег',
 '[JOB_TITLE]': 'должность_тег',
 '[TYPE_OF_ORDER]': 'тип_заказа_тег',
 '[WORK]': 'работа_тег',
 '[CERTIFICATE]': 'сертификат_тег',
 '[DEVICE]': 'устройство_тег',
 '[STANDARD]': 'стандарт_тег',
 '[REPUBLIC]': 'республика_тег',
 '[SPEED]': 'скорость_тег',
 '[НОМЕР_ДОКУМЕНТА]': 'номер_документа_тег',
 '[ADDRESS]': 'адрес_тег',
 '[ROOM]': 'комната_тег',
 '[PERIOD]': 'период_тег',
 '[MODEL]': 'модель_тег',
 '[TDU]': 'тду_тег',
 '[MINISTRY]': 'министерство_тег',
 '[BUILDING_NUMBER]': 'номер_здания_тег',
 '[KPZ]': 'кпз_тег',
 '[ORGANIZATION]': 'организация_тег',
 '[EVENT]': 'событие_тег',
 '[ORGANIZATION_ID]': 'идентификатор_организации_тег',
 '[STAGE]': 'этап_тег',
 '[POSITION]': 'позиция_тег',
 '[COPY_COUNT]': 'счет_копий_тег',
 '[PROCESSING_STANDARD]': 'стандарт_обработки_тег',
 '[EMAIL]': 'электронная_почта_тег',
 '[SNAB]': 'снаб_тег',
 '[АДРЕС_ЭЛ_ПОЧТЫ]': 'адрес_эл_почты_тег',
 '[DIVISION]': 'разделение_тег',
 '[ACTION]': 'действие_тег',
 '[MARK]': 'отметка_тег',
 '[INSTRUMENT_TYPE]': 'тип_прибора_тег',
 '[POSTAL_CODE]': 'почтовый_индекс_тег',
 '[STAND]': 'стоять_тег',
 '[VALUE]': 'ценить_тег',
 '[COPIES]': 'копии_тег',
 '[POSTCODE]': 'почтовый_индекс_тег',
 '[BANK]': 'банк_тег',
 '[STORAGE_PRICE]': 'цена_хранения_тег',
 '[OFFICE]': 'офис_тег',
 '[LETTER]': 'письмо_тег',
 '[DEPARTMENT]': 'отделение_тег',
 '[AMOUNT]': 'количество_тег',
 '[LAYER]': 'слой_тег',
 '[WIND_DIRECTION]': 'направление_ветра_тег',
 '[REGIONAL_AREA]': 'региональная_область_тег',
 '[SPECIFICATION]': 'спецификация_тег',
 '[CATEGORY]': 'категория_тег',
 '[CONFERENCING]': 'конференции_тег',
 '[PROJECT_CODE]': 'код_проекта_тег',
 '[REGULATORY_INTERACTION]': 'нормативное_взаимодействие_тег',
 '[WAREHOUSE_LIMIT]': 'складский_лимит_тег',
 '[OPERATOR]': 'оператор_тег',
 '[PRICE_PER_UNIT]': 'цена_за_единицу_тег',
 '[TOPIC]': 'тема_тег',
 '[CODE]': 'код_тег',
 '[UNIT]': 'единица_тег',
 '[WEBSITE]': 'веб-сайт_тег',
 '[OPERATION]': 'эксплуатация_тег',
 '[ДАТА]': 'данные_тег',
 '[SECTION]': 'раздел_тег',
 '[TABLE_NUMBER]': 'номер_таблицы_тег',
 '[COUNTRY]': 'страна_тег',
 '[VH_NO]': 'вх_нет_тег',
 '[LOCATION]': 'расположение_тег',
 '[CONNECTION_TYPE]': 'тип_соединения_тег',
 '[FINANCИAL_DATA]': 'финансовые_данные_тег',
 '[DIMENSION]': 'измерение_тег',
 '[SOFTWARE]': 'программное_обеспечение_тег',
 '[NOTES]': 'примечания_тег',
 '[INTERNAL_REGULATION]': 'внутреннее_правило_тег',
 '[P]': 'п_тег',
 '[ENGINE_MODEL]': 'модель_двигателя_тег',
 '[PROTOCOL]': 'протокол_тег',
 '[LICENSED_AREA]': 'лицензированная_зона_тег',
 '[INDIVIDUAL_PREMISE]': 'индивидуальное_помещение_тег',
 '[ИМЯ_ЧЕЛОВЕКА]': 'имя_человека_тег',
 '[FUNCTION]': 'функция_тег',
 '[PROSPECT]': 'перспектива_тег',
 '[SCHEME]': 'схема_тег',
 '[GAS_PIPELINE]': 'газопровод_тег',
 '[STANDARD_NUMBER]': 'стандартный_номер_тег',
 '[LAND_TYPE]': 'тип_земли_тег',
 '[INN]': 'гостиница_тег',
 '[SERVICE_CODE]': 'сервисный_код_тег',
 '[DO]': 'делать_тег',
 '[COPY]': 'копировать_тег',
 '[VEHICLE]': 'транспортное_средство_тег',
 '[INN_KPP]': 'инн_кпп_тег',
 '[SUBORGANIZATION]': 'суборганизация_тег',
 '[CONDITIONS]': 'условия_тег',
 '[REPRESENTATIVE]': 'представитель_тег',
 '[NPS_AREA]': 'зона_аэс_тег',
 '[DOCUMENT_NUMBER]': 'номер_документа_тег',
 '[PROFESSION]': 'профессия_тег',
 '[BRANCH]': 'ветвь_тег',
 '[HOUSE]': 'дом_тег',
 '[CONFERENCING_LINK]': 'ссылка_на_конференцию_тег',
 '[CENTRE]': 'центр_тег',
 '[BRAND]': 'бренд_тег',
 '[ОБЛАСТЬ]': 'область_тег',
 '[НОМЕР_ТЕЛЕФОНА]': 'номер_телефона_тег',
 '[NAME]': 'имя_тег',
 '[NOMINATION_2]': 'номинация_2_тег',
 '[REGULATORY_DOCUMENT]': 'нормативный_документ_тег',
 '[GEN_DIR]': 'общий_директор_тег',
 '[ORGANIZATION_NUMBER]': 'номер_организации_тег',
 '[PERSON]': 'человек_тег',
 '[ADMINISTRATION]': 'администрация_тег',
 '[OBJECT]': 'объект_тег',
 '[GROUP]': 'группа_тег',
 '[OKPO]': 'окпо_тег',
 '[THICKNESS]': 'толщина_тег',
 '[ROUTE]': 'маршрут_тег',
 '[PAGE_COUNT]': 'счет_страниц_тег',
 '[PRODUCT_CODE]': 'код_продукта_тег',
 '[PAGES]': 'страницы_тег',
 '[DOCUMENT]': 'документ_тег',
 '[CONDITION]': 'состояние_тег',
 '[VEHICLE_TYPE]': 'тип_автомобиля_тег',
 '[FIELD]': 'поле_тег',
 '[PHONE]': 'телефон_тег',
 '[ETHNIC_GROUP]': 'этническая_группа_тег',
 '[MANAGER]': 'менеджер_тег',
 '[RECONSTRUCTION]': 'реконструкция_тег',
 '[FINANCIAL_DIRECTOR]': 'финансовый_директор_тег',
 '[НАЗВАНИЕ_КОМПАНИИ]': 'название_компании_тег',
 '[COMPETITION_NAME]': 'название_конкурса_тег',
 '[SERVICES]': 'услуги_тег',
 '[MEASURES]': 'меры_тег',
 '[DESCRIPTION]': 'описание_тег',
 '[RESOLUTION]': 'разрешение_тег',
 '[DEPTH]': 'глубина_тег',
 '[HALL]': 'зал_тег',
 '[FINANCИАL_DATA]': 'финансовые_данные_тег',
 '[GOVERNMENT]': 'правительство_тег',
 '[CENTRAL_BANK_OF_RUSSIA]': 'центральный_банк_россии_тег',
 '[COPY_TO]': 'копировать_в_тег',
 '[COMPRESSOR_MODEL]': 'модель_компрессора_тег',
 '[ISLAND]': 'остров_тег',
 '[DIRECTION]': 'направление_тег',
 '[PRODUCT_TYPE]': 'тип_продукта_тег',
 '[MONTH]': 'месяц_тег',
 '[GENERAL_DIRECTOR]': 'генеральный_директор_тег',
 '[STATION]': 'станция_тег',
 '[POST_CODE]': 'почтовый_индекс_тег',
 '[WEB]': 'веб_тег',
 '[WORKPLACE]': 'рабочое_место_тег',
 '[OFFICE_NUMBER]': 'номер_офиса_тег',
 '[MAX_QUANTITY_ON_WAREHOUSE]': 'макс._количество_на_складе_тег',
 '[FINANCИАЛЬ_ДАННЫЕ]': 'финансовые_данные_тег',
 '[QUARTER]': 'четверть_тег',
 '[FINANCИАЛЬ_DATA]': 'финансовые_данные_тег',
 '[FINANCIAL_DATA]': 'финансовые_данные_тег',
 '[DATE_RANGE]': 'диапазон_дат_тег',
 '[YEAR+1]': 'год+1_тег',
 '[DATE_TIME]': 'дата_время_тег',
 '[DOCUMENT_TYPE]': 'тип_документа_тег',
 '[FILE_EXTENSION]': 'расширение_файла_тег',
 '[ZIP_CODE]': 'почтовый_индекс_тег',
 '[CONTACT]': 'контакт_тег',
 '[ВРЕМЯ]': 'время_тег',
 '[FILE_SIZE]': 'размер_файла_тег',
 '[MUNICIPAL_AREA]': 'муниципальный_район_тег',
 '[INSTITUTION]': 'учреждение_тег',
 '[TAX_ID]': 'налоговый_идентификатор_тег',
 '[TOTAL_PRICE]': 'общая_цена_тег'}

In [5]:
labels = [('Блок технического директора', 199),
          ('Блок директора по мощностям', 194),
          ('Блок директора по строительству', 134),
          ('Управление по проектным работам', 109),
          ('Блок заместителя генерального директора по безопасности', 100),
          ('Генеральный директор', 82),
          ('Проект "Нефтяные краюшки"', 63),
          ('Блок деректора по газу', 58),
          ('Блок заместителя генерального директора по закупкам', 54),
          ('Блок заместителя генерального директора по организационным вопросам', 46),
          ('Проект сервиса скважин', 38),
          ('Блок директора по проектированию', 35),
          ('Проект "Новая нефть"', 30),
          ('Проект "Северная деревня"', 29),
          ('Блок операционного директора', 26),
          ('Блок директора по газовым проектам', 25),
          ('Блок заместителя генерального директора по защите', 21),
          ('Блок финансового директора', 21),
          ('Блок директора по портфелю', 19),
          ('Управление землеустроительных работ', 18),
          ('Проект "Трубопроводный транспорт Главного НГКМ"', 16),
          ('Блок директора по персоналу', 14),
          ('Проект «Обустройство объектов Новой нефти»', 14),
          ('Проект "Восточный"', 14),
          ('Блок заместителя генерального директора по имуществу', 14),
          ('Проект "Южный"', 10),
          ('Блок бизнес-директора', 6),
          ('Управление коммуникаций', 6),
          ('Проект «Обустройство Интересного лицензионного участка»', 6),
          ('Блок исполнительного директора по реализации проекта "Большое месторождение"',
           5),
          ('Проект "Обустройство площадных объектов НГКМ Поменбше"', 5),
          ('Подразделение по информационным технологиям', 2),
          ('Имущественные вопросы', 2),
          ('Проект «Трубопроводный транспорт Ещё одного НГКМ»', 2),
          ('Блок заместителя генерального директора по строительству', 2),
          ('Проект «Обустройство объектов Новейшей нейти»', 2)]

In [6]:
def replace_tags_in_text(text: str):
    for tag in tag_dict:
        text = text.replace(tag, tag_dict[tag])
    return text

def replace_tags(df):
    df_copy = df.copy()
    
    df_copy['text'] = df_copy['text'].astype(str).apply(replace_tags_in_text)
    
    return df_copy

In [6]:
tag_dict_rev = {}
for i in tag_dict:
    tag_dict_rev[tag_dict[i]] = i
    
tag_dict_rev

def replace_tags_in_text_rev(text: str):
    for tag in tag_dict_rev:
        text = text.replace(tag, tag_dict_rev[tag])
    return text

def replace_tags_rev(df):
    df_copy = df.copy()
    
    df_copy['text'] = df_copy['text'].astype(str).apply(replace_tags_in_text_rev)
    
    return df_copy

df = pd.read_csv('augmented_only_trans.csv')
df = replace_tags_rev(df)
df.to_csv('augmented_trans_and_shuffle_raw_tags.csv')

In [7]:
def low(df):
    df_copy = df.copy()
    # Заполняем NaN пустой строкой и приводим всё к str, затем в нижний регистр
    df_copy['text'] = df_copy['text'].fillna('').astype(str).str.lower()
    return df_copy

In [8]:
def LLM_gen_new_text(text: str) -> str:
    client = OpenAI(base_url="http://localhost:1234/v1", api_key="not-needed")

    system_prompt = 'Твоя задача - аугментировать тексты для обучения многоклассового классификатора. Ты получаешь на вход текст и должен вернуть только новый текст на основе предыдущего. Ты можешь заменять слова синонимами, менять местами предложения, перефразировать, но смысл и стиль текста должен оставаться неизменным. Нельзя создавать новые слова с подстрокой "_тег". Текст должен быть написан одним абзацам, без переноса строки или перечислений списков. Лексическое наполнение полученного текста должны быть максимально приближенны к оригинальному. Делай упор на замену слов на синонимы.'

    response = client.chat.completions.create(
        model="qwen/qwen3-vl-4b",
        max_tokens=16384,
        messages=[{"role": "system", "content": system_prompt},
                  {"role": "user", "content": text}],
        seed=random.randint(1, 100000000000)
    )
    
    return response.choices[0].message.content

def LLM_augmentation(df):
    df_copy = df.copy()
    for i in range(len(df_copy)):
        if 10 < list(filter(lambda x: x[0] == df_copy['label'][i], labels))[0][1] < 30:
            new_row = df_copy.iloc[i].copy()
            new_row['idx'] = f'{new_row['idx']}_LLM_1'
            # print(new_row['idx'])
            try:
                new_row['text'] = LLM_gen_new_text(new_row['text'])
            except Exception as e:
                print(e)
                print(i)
                print(new_row['idx'])
                continue
            
            df_copy = pd.concat([df_copy, pd.DataFrame([new_row])], ignore_index=True)
            
        elif 4 < list(filter(lambda x: x[0] == df_copy['label'][i], labels))[0][1] <= 10:
            new_row1 = df_copy.iloc[i].copy()
            new_row2 = df_copy.iloc[i].copy()
            new_row3 = df_copy.iloc[i].copy()
            new_row1['idx'] = f'{new_row1['idx']}_LLM_1'
            new_row2['idx'] = f'{new_row2['idx']}_LLM_2'
            new_row3['idx'] = f'{new_row3['idx']}_LLM_3'
            # print(new_row['idx'])
            try:
                new_row1['text'] = LLM_gen_new_text(new_row1['text'])
                new_row2['text'] = LLM_gen_new_text(new_row2['text'])
                new_row3['text'] = LLM_gen_new_text(new_row3['text'])
            except Exception as e:
                print(e)
                print(i)
                print(new_row1['idx'])
                continue
            
            df_copy = pd.concat([df_copy, pd.DataFrame([new_row1])], ignore_index=True)
            df_copy = pd.concat([df_copy, pd.DataFrame([new_row2])], ignore_index=True)
            df_copy = pd.concat([df_copy, pd.DataFrame([new_row3])], ignore_index=True)
            
        elif list(filter(lambda x: x[0] == df_copy['label'][i], labels))[0][1] <= 4:
            new_row1 = df_copy.iloc[i].copy()
            new_row2 = df_copy.iloc[i].copy()
            new_row3 = df_copy.iloc[i].copy()
            new_row4 = df_copy.iloc[i].copy()
            new_row5 = df_copy.iloc[i].copy()
            new_row1['idx'] = f'{new_row1['idx']}_LLM_1'
            new_row2['idx'] = f'{new_row2['idx']}_LLM_2'
            new_row3['idx'] = f'{new_row3['idx']}_LLM_3'
            new_row4['idx'] = f'{new_row4['idx']}_LLM_4'
            new_row5['idx'] = f'{new_row5['idx']}_LLM_5'
            # print(new_row['idx'])
            try:
                new_row1['text'] = LLM_gen_new_text(new_row1['text'])
                new_row2['text'] = LLM_gen_new_text(new_row2['text'])
                new_row3['text'] = LLM_gen_new_text(new_row3['text'])
                new_row4['text'] = LLM_gen_new_text(new_row4['text'])
                new_row5['text'] = LLM_gen_new_text(new_row5['text'])
            except Exception as e:
                print(e)
                print(i)
                print(new_row1['idx'])
                continue
            
            df_copy = pd.concat([df_copy, pd.DataFrame([new_row1])], ignore_index=True)
            df_copy = pd.concat([df_copy, pd.DataFrame([new_row2])], ignore_index=True)
            df_copy = pd.concat([df_copy, pd.DataFrame([new_row3])], ignore_index=True)
            df_copy = pd.concat([df_copy, pd.DataFrame([new_row4])], ignore_index=True)
            df_copy = pd.concat([df_copy, pd.DataFrame([new_row5])], ignore_index=True)
            
    return df_copy
            
# LLM_gen_new_text('Российская Федерация ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ [ORGANIZATION] [LOCATION] тел.факс: [CONTACT] E-mail: [CONTACT] Исх. No [DOCUMENT_NUMBER] [PERSON] Уважаемый [PERSON]! В ответ на Ваше письмо от [DATE_TIME] О возможности заключения ДСNo2, сообщаю Вам, что [ORGANIZATION] в соответствии с договором No [DOCUMENT_NUMBER] от [DATE_TIME] выполняет работы по предотвращению и ликвидации АСПО и гидратных отложений в скважине физическими методами для скважин [OBJECT] и выражает согласие на подписание Дополнительного соглашения No2. С Уважением, Директор [ORGANIZATION] [PERSON] Исполнитель: [PERSON] ([CONTACT]) [ORGANIZATION], ООО Вх. No [DOCUMENT_NUMBER] от [DATE_TIME]')

In [9]:
def clean_text_spaces(df):
    """
    Удаляет все двойные пробелы, переносы строк и табуляции из столбца 'text' DataFrame
    
    Parameters:
    df (pd.DataFrame): Входной DataFrame
    
    Returns:
    pd.DataFrame: DataFrame с очищенным столбцом 'text'
    """
    # Создаем копию DataFrame, чтобы не изменять оригинал
    result_df = df.copy()
    
    # Применяем замены к столбцу 'text'
    result_df['text'] = result_df['text'].astype(str).apply(
        lambda x: re.sub(r'\s+', ' ', x).strip()
    )
    
    return result_df

In [10]:
blacklist = [i for i in tag_dict]

LIMIT = 1000

def translate_text(count: int, base_text: str) -> list:
    results = []
    
    # Заменяем слова из блеклиста, чтобы переводчик их не менял
    for i in range(len(blacklist)):
        base_text = base_text.replace(blacklist[i], f'__{i}__')
    
    # Если длина текса больше LIMIT символов - разбиваем на куски
    if len(base_text) > LIMIT:
        sub_str_count = (len(base_text) + LIMIT - 1) // LIMIT
        new_texts = ['' for _ in range(count)]
        exploded_text = [base_text[i:i + LIMIT] for i in range(0, len(base_text), LIMIT)]
        for i in range(sub_str_count-1):
            while exploded_text[i][-1] != '.':
                if exploded_text[i].count('.') == 0 and exploded_text[i][-1] == ' ':
                    break
                exploded_text[i+1] = exploded_text[i][-1] + exploded_text[i+1]
                exploded_text[i] = exploded_text[i][:-1]
        for i in range(sub_str_count):
            new_sub_strs = [i for i in translate_text(count, exploded_text[i])]
            for j in range(count):
                new_texts[j] += ' ' + new_sub_strs[j]
    
        for i in range(count):
            results.append(new_texts[i])
            
        return results
        
    # список языков для случайного перевода (можно менять)
    languages = ['en', 'de', 'fr', 'es', 'it']
    # languages = ['english', 'german', 'french', 'spanish', 'italian']
    for i in range(count):
        # выбираем случайный язык для “обратного перевода”
        lang1 = random.choice(languages)
        # lang2 = random.choice(languages)

        new_text = GoogleTranslator(source='ru', target=lang1).translate(base_text)
        print(123)
        
        # new_text = GoogleTranslator(source=lang1, target=lang2).translate(base_text)
        
        new_text = GoogleTranslator(source=lang1, target='ru').translate(base_text)
        print(456)
        
        time.sleep(1)
            
        # Возвращаем слова из блеклиста
        for i in range(len(blacklist)):
            new_text = new_text.replace(f'__{i}__', blacklist[i]).replace(f'__{i}_', blacklist[i]).replace(f'_{i}__', blacklist[i]).replace(f'_{i}_', blacklist[i])
        
        results.append(new_text)
    
    return results


def translate(df):
    df_copy = df.copy()
    for i in range(len(df_copy)):
        if '_LLM' in df_copy['idx'][i]:
            continue
        label_count = list(filter(lambda x: x[0] == df_copy['label'][i], labels))[0][1]
        
        if label_count == 2:
            translate_count = 2
        elif label_count <= 6:
            translate_count = 1
        elif label_count <= 14:
            translate_count = random.choice([0, 1])
        else:
            translate_count = 0
            
        if translate_count == 0:
            continue
        
        try: 
            new_texts = translate_text(translate_count, df_copy['text'][i])
        except Exception as e:
            print(e)
            print(i)
            print(new_row['idx'])
            continue
            
        for j in range(translate_count):
            new_row = df_copy.iloc[i].copy()
            new_row['idx'] = f'{new_row['idx']}_translate_{j+1}'
            new_row['text'] = new_texts[j]
            
            df_copy = pd.concat([df_copy, pd.DataFrame([new_row])], ignore_index=True)
            
    return df_copy

# translate_text(1, '[ORGANIZATION] [LOCATION] Тел.: [CONTACT], факс: [CONTACT] e-mail: [CONTACT] www.[ORGANIZATION] ОКПО [ID], ОГРН [DOCUMENT_NUMBER], ИНН [ID] КПП [ID] [DATE_TIME] No [DOCUMENT_NUMBER] на [DOCUMENT_NUMBER] от [DATE_TIME] О согласовании размещения объекта ([OBJECT] от [ORGANIZATION] на [OBJECT]) Уважаемый [PERSON]! В ответ на письмо от [DATE_TIME] No [DOCUMENT_NUMBER] [ORGANIZATION] (далее - [ORGANIZATION]), являясь владельцем лицензии на право пользования недрами от [DATE_TIME] [ID], согласовывает оформление разрешительной документации на право пользования земельными участками под размещение объекта [OBJECT] в границах горного отвода [OBJECT] согласно схемы, прилагаемой к настоящему письму при выполнении следующих условий: - гарантии безусловного перевода права пользования участками, не задействованными в строительстве [ORGANIZATION], на [ORGANIZATION] и [ORGANIZATION] при необходимости их использования для реализации инвестиционных проектов; - получения положительного согласования [ORGANIZATION]; - получение и неукоснительное исполнение технических условий при пересечении действующих и проектируемых коммуникаций [ORGANIZATION] и [ORGANIZATION]; - неукоснительного соблюдения требований земельного и природоохранного законодательства. Приложение: Обзорная схема границ ЗУ на 1 л. [PERSON] [PERSON] [CONTACT] [NUMBER] Заместитель генерального директора по ремонту и капитальному строительству [PERSON] ДОКУМЕНТ ПОДПИСАН ЭЛЕКТРОННОЙ ПОДПИСЬЮ Сертификат [ID] Владелец [PERSON] Действителен с [DATE_TIME] по [DATE_TIME] Доверенность No [DOCUMENT_NUMBER] Обзорная схема расположения земельных участков испрашиваемых [ORGANIZATION] под размещение объекта: [OBJECT] Узел А Масштаб 1:1500 Условные обозначения - Граница ЗУ согласно сведений ЕГРН; - Граница испрашиваемых ЗУ под размещение объектов [OBJECT]; - Промышленные площадки и кусты скважин [OBJECT]; - автомобильные дороги с твердым покрытием; - автомобильные дороги грунтовые; - кусты газовых скважин; - промышленные площадки; - линии электропередач; - трубопроводы; - газопроводы; - метанолопроводы; - пункты опорной маркшейдерской сети. Составил: ведущий маркшейдер СГМ [ORGANIZATION] [PERSON] [DATE_TIME]г. Масштаб 1:1 Формат А3 [ORGANIZATION], ООО Вх. No [DOCUMENT_NUMBER] от [DATE_TIME]')

In [11]:
def shuffle(text):
    sents = re.split(r'(?<=[.!?])\s+', text)
    if len(sents) > 1:
        random.shuffle(sents)
        return " ".join(sents)
    return text

In [12]:
def translate2(df):
    df_copy = df.copy()
    for i in range(len(df_copy)):
        print(i, df_copy['idx'][i])
        if '_LLM' in df_copy['idx'][i]:
            continue
        label_count = list(filter(lambda x: x[0] == df_copy['label'][i], labels))[0][1]
        
        augm_count = 20 / label_count
        
        augm_count = random.randint(int(augm_count*100), int((augm_count + 1)*100)) // 100
            
        if augm_count == 0 or label_count > 20:
            continue
        elif augm_count > 5:
            augm_count = 5
            
        trans_shuffle = [0, 0]
        
        for j in range(augm_count):
            trans_shuffle[random.choice([0, 0, 0, 0, 1])] += 1
        
        new_texts = []
        try: 
            new_texts = [[j, 0] for j in translate_text(trans_shuffle[0], df_copy['text'][i])]
            for j in range(trans_shuffle[1]):
                new_texts.append([shuffle(df_copy['text'][i]), 1])
        except Exception as e:
            print(e)
            print(i)
            print(new_row['idx'])
            continue
            
        for j in range(augm_count):
            new_row = df_copy.iloc[i].copy()
            if new_texts[j][1]:
                new_row['idx'] = f'{new_row['idx']}_shuffle_{j+1}'
            else:
                new_row['idx'] = f'{new_row['idx']}_translate_{j+1}'
            new_row['text'] = new_texts[j][0]
            
            df_copy = pd.concat([df_copy, pd.DataFrame([new_row])], ignore_index=True)
            
    return df_copy

In [ ]:
# df = replace_tags(df)
# df = low(df)
# df = clean_text_spaces(df)

# df
# df.to_csv('raw_precleaned.csv')

df = clean_text_spaces(df)
# df = LLM_augmentation(df)
df = translate2(df)
# df = replace_tags(df)
# df = low(df)
# df = clean_text_spaces(df)

df.to_csv('augmented_trans_and_shuffle_raw_tags.csv')

0 10026
1 1005
2 1010
3 1013
4 10186
5 10236
6 10282
7 10543
8 1078
9 1088
10 1101
11 1112


In [14]:
a = set()

for i in range(len(df)):
    if type(df['idx'][i]) == float:
        print(i, df['idx'][i], df['text'][i])
    a.add(type(df['idx'][i]))
    
a
# df[df['idx'] == 'NaN']

{str}

Error code: 400 - {'error': 'Reached context length of 16384 tokens with model (arch: qwen3vl) that does not currently support mid-generation context overflow. Try reloading with a larger context length or shortening the prompt/chat.'}
112
1266_LLM_1
Error code: 400 - {'error': 'Reached context length of 16384 tokens with model (arch: qwen3vl) that does not currently support mid-generation context overflow. Try reloading with a larger context length or shortening the prompt/chat.'}
175
1347_LLM_1
Error code: 400 - {'error': 'Reached context length of 16384 tokens with model (arch: qwen3vl) that does not currently support mid-generation context overflow. Try reloading with a larger context length or shortening the prompt/chat.'}
777
6250_LLM_1
Error code: 400 - {'error': 'Reached context length of 16384 tokens with model (arch: qwen3vl) that does not currently support mid-generation context overflow. Try reloading with a larger context length or shortening the prompt/chat.'}
795
6342_LLM_1
Error code: 400 - {'error': 'Reached context length of 16384 tokens with model (arch: qwen3vl) that does not currently support mid-generation context overflow. Try reloading with a larger context length or shortening the prompt/chat.'}
951
7076_LLM_1
Error code: 400 - {'error': 'Reached context length of 16384 tokens with model (arch: qwen3vl) that does not currently support mid-generation context overflow. Try reloading with a larger context length or shortening the prompt/chat.'}
1118
6288_LLM_1
Error code: 400 - {'error': 'Reached context length of 16384 tokens with model (arch: qwen3vl) that does not currently support mid-generation context overflow. Try reloading with a larger context length or shortening the prompt/chat.'}
1189
6521_LLM_1
Error code: 400 - {'error': 'Reached context length of 16384 tokens with model (arch: qwen3vl) that does not currently support mid-generation context overflow. Try reloading with a larger context length or shortening the prompt/chat.'}
1201
6571_LLM_1
Error code: 400 - {'error': 'Reached context length of 16384 tokens with model (arch: qwen3vl) that does not currently support mid-generation context overflow. Try reloading with a larger context length or shortening the prompt/chat.'}
1207
6599_LLM_1

In [15]:
df['text'][112]

'организация_тег организация_тег дивизион область_тег заместитель генерального директора по закупкам организация_тег человек_тег юридический адрес: расположение_тег адрес для корреспонденции: расположение_тег тел.: контакт_тег e-mail: контакт_тег окпо идентификатор_тег, огрн идентификатор_тег, инн идентификатор_тег, кпп идентификатор_тег no на no от объект_тег пдн уважаемый человек_тег! в целях исполнения поручений по организации хранения мтр, для обеспечения проектов область_тег, просим рассмотреть возможность реализации в адрес организация_тег тмц (плиты дорожная пдн м-атvi,55 6000x2000x140мм) в количестве до финансовые_данные_тег шт. со складов храненияnoc. город_тег. в случае отсутствия возможности реализации плиты, находящейся на балансе организация_тег, просим рассмотреть вариант займа с последующим возвратом в дата_время_тег. с уважением, директор дивизиона область_тег человек_тег исп: человек_тег тел.: контакт_тег документ подписан усиленной кавалифицированной электронной подпи

In [16]:
'[ORGANIZATION] НЕФТЬ Публичное акционерное общество [ORGANIZATION] НЕФТЬ (ПАО [ORGANIZATION] НЕФТЬ) Для корреспонденции: [LOCATION] 190121 тел.: [CONTACT] факс: [CONTACT] e-mail: [CONTACT], www.[ORGANIZATION] НЕФТЬ.ru [ID], [ID], [ID], [ID] [DATE_TIME] No [DOCUMENT_NUMBER] на No от Об участии в семинаре для авторов Мобильной ленты Уважаемые коллеги! [DATE_TIME] в [LOCATION] (Цифровой дом Циферграуз, набережная Адмиралтейского канала, дом 2, литера И) состоится обучающий семинар для авторов каналов в корпоративной мобильной информационной сети Мобильная лента. Прошу командировать на семинар сотрудников согласно приложенному списку. Повестка мероприятия: 1. Презентация возможностей Мобильной ленты для решения задач бизнеса; 2. Обзор успешных практик использования платформы в коммуникационных целях; 3. Мастер-класс Как вести блог с помощью нейросетей; 4. Сбор предложений по развитию Мобильной ленты в 2025 году. Приложение: список участников совещания на 1 л. в 1 экз. С уважением, Член Правления, начальник департамента по корпоративным коммуникациям и региональному развитию [PERSON] Исполнитель: [PERSON] [CONTACT] [PERSON] Приложение К письму No От Список рассылки No [PERSON] [POSITION] 1. [PERSON] [POSITION], [ORGANIZATION] [ORGANIZATION] 2. [PERSON] [POSITION], [ORGANIZATION] [ORGANIZATION] 3. [PERSON] [POSITION], [ORGANIZATION] [ORGANIZATION] 4. [PERSON] [POSITION], [ORGANIZATION] [ORGANIZATION] 5. [PERSON] [POSITION], [ORGANIZATION] [ORGANIZATION] 6. [PERSON] [POSITION], [ORGANIZATION] [ORGANIZATION] 7. [PERSON] [POSITION], [ORGANIZATION] [ORGANIZATION] 8. [PERSON] [POSITION], [ORGANIZATION] [ORGANIZATION] 9. [PERSON] [POSITION], [ORGANIZATION] [ORGANIZATION] 10. [PERSON] [POSITION], [ORGANIZATION] [ORGANIZATION] 11. [PERSON] [POSITION], [ORGANIZATION] [ORGANIZATION] 12. [PERSON] [POSITION], [ORGANIZATION] [ORGANIZATION] 13. [PERSON] [POSITION], [ORGANIZATION] [ORGANIZATION] 14. [PERSON] [POSITION], [ORGANIZATION] [ORGANIZATION] 15. [PERSON] [POSITION], [ORGANIZATION] [ORGANIZATION]'

'[ORGANIZATION] НЕФТЬ Публичное акционерное общество [ORGANIZATION] НЕФТЬ (ПАО [ORGANIZATION] НЕФТЬ) Для корреспонденции: [LOCATION] 190121 тел.: [CONTACT] факс: [CONTACT] e-mail: [CONTACT], www.[ORGANIZATION] НЕФТЬ.ru [ID], [ID], [ID], [ID] [DATE_TIME] No [DOCUMENT_NUMBER] на No от Об участии в семинаре для авторов Мобильной ленты Уважаемые коллеги! [DATE_TIME] в [LOCATION] (Цифровой дом Циферграуз, набережная Адмиралтейского канала, дом 2, литера И) состоится обучающий семинар для авторов каналов в корпоративной мобильной информационной сети Мобильная лента. Прошу командировать на семинар сотрудников согласно приложенному списку. Повестка мероприятия: 1. Презентация возможностей Мобильной ленты для решения задач бизнеса; 2. Обзор успешных практик использования платформы в коммуникационных целях; 3. Мастер-класс Как вести блог с помощью нейросетей; 4. Сбор предложений по развитию Мобильной ленты в 2025 году. Приложение: список участников совещания на 1 л. в 1 экз. С уважением, Член Пр

Error code: 400 - {'error': 'Reached context length of 4096 tokens with model (arch: qwen3vl) that does not currently support mid-generation context overflow. Try reloading with a larger context length or shortening the prompt/chat.'}
114
1268_LLM_1
Error code: 400 - {'error': 'Reached context length of 4096 tokens with model (arch: qwen3vl) that does not currently support mid-generation context overflow. Try reloading with a larger context length or shortening the prompt/chat.'}
138
1303_LLM_1
Error code: 400 - {'error': 'Reached context length of 4096 tokens with model (arch: qwen3vl) that does not currently support mid-generation context overflow. Try reloading with a larger context length or shortening the prompt/chat.'}
139
1304_LLM_1
Error code: 400 - {'error': 'Reached context length of 4096 tokens with model (arch: qwen3vl) that does not currently support mid-generation context overflow. Try reloading with a larger context length or shortening the prompt/chat.'}
795
6342_LLM_1
Error code: 400 - {'error': 'Reached context length of 4096 tokens with model (arch: qwen3vl) that does not currently support mid-generation context overflow. Try reloading with a larger context length or shortening the prompt/chat.'}
1088
6170_LLM_1
Error code: 400 - {'error': 'Reached context length of 4096 tokens with model (arch: qwen3vl) that does not currently support mid-generation context overflow. Try reloading with a larger context length or shortening the prompt/chat.'}
1118
6288_LLM_1
Error code: 400 - {'error': 'Reached context length of 4096 tokens with model (arch: qwen3vl) that does not currently support mid-generation context overflow. Try reloading with a larger context length or shortening the prompt/chat.'}
1207
6599_LLM_1
Error code: 400 - {'error': 'Reached context length of 4096 tokens with model (arch: qwen3vl) that does not currently support mid-generation context overflow. Try reloading with a larger context length or shortening the prompt/chat.'}
1260
6906_LLM_1

In [17]:
import re
import pandas as pd
from collections import Counter

# === 1. Настройки ===
TEXT_COL = "text"
LABEL_COL = "label"

# Символы, которые встречаются редко и вызывают подозрение (единичные из твоей статистики)
suspect_chars = ['©', '#', '+', '*', '“', '”', '„', '—', '−', '%', '°', '№', 'ә']

# === 2. Подсчёт частоты подозрительных символов ===
def count_suspects_by_label(df):
    stats = []
    for ch in suspect_chars:
        mask = df[TEXT_COL].str.contains(re.escape(ch))
        df_ch = df[mask]
        label_counts = df_ch[LABEL_COL].value_counts()
        stats.append({
            'char': ch,
            'total_occurrences': len(df_ch),
            'unique_labels': len(label_counts),
            'top_label': label_counts.index[0] if len(label_counts) > 0 else None,
            'top_label_count': label_counts.iloc[0] if len(label_counts) > 0 else 0
        })
    return pd.DataFrame(stats).sort_values("total_occurrences", ascending=False)

df_symbol_stats = count_suspects_by_label(df)

In [18]:
# === 3. Поиск иностранных вставок (латиница, азиатские символы, непонятные куски) ===
def detect_foreign_snippets(df):
    foreign_pattern = re.compile("[^а-яА-Яa-zA-zёЁ0-9\\d\\s\\n.,;:!?()_\\/'\\-*]")
    df_foreign = df[df[TEXT_COL].str.contains(foreign_pattern)]
    return df_foreign[['idx', LABEL_COL, TEXT_COL]]

df_foreign = detect_foreign_snippets(df)

In [19]:
# === 4. Логика принятия решения ===
def classify_symbol_row(row):
    if row["total_occurrences"] <= 2 and row["unique_labels"] >= 3:
        return "🗑 удалить (реальный шум)"
    elif row["unique_labels"] <= 2 and row["total_occurrences"] > 1:
        return "✅ оставить (признак класса)"
    else:
        return "🤔 проверить вручную"

df_symbol_stats["decision"] = df_symbol_stats.apply(classify_symbol_row, axis=1)

In [20]:
# === 5. Вывод ===
print("=== Анализ подозрительных символов ===")
print(df_symbol_stats.to_string(index=False))
print("\n=== Подозрительные тексты с иностранными вставками ===")
print(f"Найдено: {len(df_foreign)} примеров\n")

for i, row in df_foreign.head(5).iterrows():
    print(f"→ {row[LABEL_COL]} {row['idx']}\n{row[TEXT_COL][:400]}\n{'-'*60}")

# === 6. Рекомендация для удаления ===
symbols_to_remove = df_symbol_stats.query("decision == '🗑 удалить (реальный шум)'")["char"].tolist()
print("\nСимволы, которые можно удалить без потери сигналов класса:", symbols_to_remove)

=== Анализ подозрительных символов ===
char  total_occurrences  unique_labels                                               top_label  top_label_count                    decision
   №                211             20         Проект "Трубопроводный транспорт Главного НГКМ"               21         🤔 проверить вручную
   *                199             26                             Блок технического директора               30         🤔 проверить вручную
   —                 10              6                                   Блок бизнес-директора                4         🤔 проверить вручную
   ә                  5              5     Блок заместителя генерального директора по закупкам                1         🤔 проверить вручную
   %                  2              2                             Блок директора по мощностям                1 ✅ оставить (признак класса)
   ”                  1              1                                    Генеральный директор                1         🤔